In [10]:
import requests
import json
import pyarrow as pa
import base64

In [19]:


# Define the API URL
url = "http://127.0.0.1:5000/create"
schema = pa.schema([
    pa.field("disclaimer", pa.string()),
    pa.field("file_path", pa.string()),
    pa.field("n_transitions", pa.int32()),
    pa.field("success", pa.bool_()),  # Corrected boolean field
    pa.field("success_labeled_by", pa.string())
])

serialized_schema = schema.serialize().to_pybytes()
encoded_schema = base64.b64encode(serialized_schema).decode('utf-8')

# Define the payload
data = {
    "dataset": "ucsd_pick_and_place_dataset_converted_externally_to_rlds", # create a dataset table with this name
    "uri": "/Users/haoweichung/Development/CMU/Spring2024/FogROS2-RTX/FogX-Store/_datasets/dataset_db", # load lancedb from this uri
    "schema" : encoded_schema # user defined data schema
}

# Send the POST request
response = requests.post(url, json=data)

# Print the response
print(response.status_code)
print(response.json())


200
{'message': 'Database created successfully'}


In [20]:
import requests

# Define the API URL
url = "http://127.0.0.1:5000/write"

# Define the payload
data = {
    "ds_path": "gs://gresearch/robotics/ucsd_pick_and_place_dataset_converted_externally_to_rlds/0.1.0", # local or remote rlds dataset location
    "dataset" : "ucsd_pick_and_place_dataset_converted_externally_to_rlds", # The dataset to append new data
    "uri": "/Users/haoweichung/Development/CMU/Spring2024/FogROS2-RTX/FogX-Store/_datasets/dataset_db" # lancedb uri
}

# Send the POST request
response = requests.post(url, json=data)

# Print the response
print(response.status_code)
print(response.json())


200
{'message': 'Dataset written successfully'}


# Let's read some data from the database

In [21]:
import lancedb
uri = "/Users/haoweichung/Development/CMU/Spring2024/FogROS2-RTX/FogX-Store/_datasets/dataset_db"
db = lancedb.connect(uri)
tbl = db.open_table("ucsd_pick_and_place_dataset_converted_externally_to_rlds")

In [22]:
tbl.to_arrow()

pyarrow.Table
disclaimer: string
file_path: string
n_transitions: int32
success: bool
success_labeled_by: string
----
disclaimer: [["b'reward is noisy for this dataset partition'"],["b'none'"],...,["b'reward is noisy for this dataset partition'"],["b'reward is noisy for this dataset partition'"]]
file_path: [["b'data/train/tabletop_uncurated/209.p'"],["b'data/train/tabletop_base/515.p'"],...,["b'data/train/tabletop_uncurated/533.p'"],["b'data/train/tabletop_uncurated/76.p'"]]
n_transitions: [[50],[50],...,[50],[50]]
success: [[true],[false],...,[true],[true]]
success_labeled_by: [["b'classifier'"],["b'classifier'"],...,["b'classifier'"],["b'classifier'"]]

# Select all the data with success is true

In [23]:
result = (
    tbl.search()
    .where("success = true", prefilter=True)
    .to_arrow()
)

In [24]:
result

pyarrow.Table
disclaimer: string
file_path: string
n_transitions: int32
success: bool
success_labeled_by: string
----
disclaimer: [["b'reward is noisy for this dataset partition'"],["b'none'"],...,["b'reward is noisy for this dataset partition'"],["b'reward is noisy for this dataset partition'"]]
file_path: [["b'data/train/tabletop_uncurated/209.p'"],["b'data/train/tabletop_base/157.p'"],...,["b'data/train/tabletop_uncurated/533.p'"],["b'data/train/tabletop_uncurated/76.p'"]]
n_transitions: [[50],[50],...,[50],[50]]
success: [[true],[true],...,[true],[true]]
success_labeled_by: [["b'classifier'"],["b'classifier'"],...,["b'classifier'"],["b'classifier'"]]